## Welcome to a DIY Wrapped Project!

This notebook is your playground for building a personalized "Wrapped" campaign, just like Spotify Wrapped, but with your own data twist! Whether you’re into gaming, reading, or coding, this short tutorial will show you how to make it yours.

This started as a small personal project to explore building simple wrapped campaigns in Python, and I realized it might make for an interesting starter project.

- **Two Ways to Explore**:
  1. Run this notebook step-by-step (great for learning!).
  2. Use the Python file for a standalone app (faster if you’re ready to roll).
- **What’s Ahead**: We’ll generate personal data, create an interactive app, and add some AI capabilities—all in Python.

## Phase 1 Prep: Setting Up the Tools

Before we dive into the fun stuff, we need to grab Dash—a super cool Python library for building interactive web apps.

- **`!pip install dash`**: This command uses `pip` (Python’s package installer) to download Dash. The `!` runs it as a shell command in Jupyter.
- **Why We Need It**: Dash powers our interactive visuals in Phase 2. No Dash, no flipping through cool charts!
- **Good to Know**: On Google Colab, we run this every time to ensure Dash is fresh. Locally, you might only install it once.

In [3]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


## Phase 1: Crafting Your Personalized Data

Here’s where it starts! We’ll create a small dataset tailored to you—your name, your favorite activity (like Coding or Gaming), and some fake stats over 5 years. This is the heart of your Wrapped story.

- **Imports**: `numpy` (np) for math and random numbers, `pandas` (pd) for organizing data into tables.
- **`generate_personalized_data`**:
  - Takes your `name` and `category` as inputs.
  - Uses `np.random.normal` to make fake activity data with a slight upward trend (`+ (years - 2018) * 0.5`)—like you’re getting better over time!
  - Builds a table (DataFrame) with years, activity, category, and your name.
- **How It Works**: You type your name and category, and we print + save the data to `personalized_data.csv`.
- **Why It’s Cool**: This is your data—no one else has it! It’s the raw material for our Wrapped app.

In [1]:
import numpy as np
import pandas as pd

# Function to generate personalized data
def generate_personalized_data(name, category):
    np.random.seed(42)  # For reproducibility
    years = np.arange(2018, 2023)  # 5 years of data
    # Simulate activity with a slight upward trend + random noise
    activity = np.random.normal(loc=10, scale=2, size=len(years)) + (years - 2018) * 0.5
    # Create a DataFrame
    df = pd.DataFrame({
        'Year': years,
        'Activity': activity,
        'Category': [category] * len(years),
        'User': [name] * len(years)
    })
    return df

# Example usage
name = input("Enter your name: ")
category = input("Enter a category (e.g., Gaming, Reading, Coding): ")
df_personalized = generate_personalized_data(name, category)
print(f"\nHere’s your personalized data for {category}, {name}!")
print(df_personalized)

# Save to CSV for later use
df_personalized.to_csv("personalized_data.csv", index=False)

Enter your name: Star
Enter a category (e.g., Gaming, Reading, Coding): Coding

Here’s your personalized data for Coding, Star!
   Year   Activity Category  User
0  2018  10.993428   Coding  Star
1  2019  10.223471   Coding  Star
2  2020  12.295377   Coding  Star
3  2021  14.546060   Coding  Star
4  2022  11.531693   Coding  Star


## Phase 2 & 3: Interactive Wrapped + AI Magic

Now we bring it all together! Phase 2 builds an interactive app to flip through your Wrapped story, and Phase 3 sprinkles in a smart AI insight.

### Phase 2: The Interactive App
- **Imports**: `dash` for the app, `plotly.express` (px) for charts, `pandas` to load your data.
- **Data**: Loads `personalized_data.csv` and grabs your name and category.
- **`steps` List**: Defines 4 steps—welcome text, bar chart, pie chart, and goodbye text—personalized with your info.
- **Layout**:
  - `dcc.Store` tracks which step you’re on (like a bookmark).
  - `step-container` shows the current step; buttons let you flip forward/backward.
- **Callback (`update_step`)**:
  - Listens to "Next" and "Previous" clicks.
  - Updates the step and shows either text or a plot (bar for trends, pie for highlights).
- **Plots**: Made with Plotly—colorful, interactive, and tied to your data.

### Phase 3: Adding AI Personalization
- **`get_insight`**: A small function to identify your peak year and activity, and add a bit of personalization.
- **New Step**: Adds this insight as a final text card—e.g., “Hey Star, your Coding peaked in 2021!”

### Running It
- `app.run(debug=True)` launches the app. On Colab, you’ll see a link; locally, visit `http://127.0.0.1:8050/`.
- **Heads Up**: Run Phase 1 first to make the CSV, or this won’t work!

In [11]:
import dash
from dash import html, dcc, Input, Output, State, callback
import plotly.express as px
import pandas as pd

# Load personalized data from Phase 1
df = pd.read_csv("personalized_data.csv")
user_name = df['User'].iloc[0]  # Get user's name
category = df['Category'].iloc[0]  # Get user's category

app = dash.Dash(__name__)

# Define the steps for the Wrapped campaign
steps = [
    {
        'type': 'text',
        'title': f'🔮 Welcome, {user_name}! 🔮',
        'content': f'Here’s your {category} Wrapped for 2018-2022!'
    },
    {
        'type': 'plot',
        'title': f'Your {category} Trend',
        'content': f'Check out how your {category} activity changed over time.'
    },
    {
        'type': 'plot',
        'title': 'Yearly Highlights',
        'content': f'Here’s a pie chart of your {category} activity distribution.'
    }
]

# Use a dcc.Store to persist the current step across callbacks (better than global variable)
app.layout = html.Div(
    style={
        'display': 'flex', 'flex-direction': 'column', 'align-items': 'center',
        'width': '500px', 'margin': '20px auto', 'font-family': 'Arial, sans-serif'
    },
    children=[
        dcc.Store(id='step-store', data=0),  # Store current step
        html.Div(id='step-container', children=[]),
        html.Div(
            style={'display': 'flex'},
            children=[
                html.Button("Previous", id='prev-button', n_clicks=0,
                            style={'margin-top': '20px', 'background-color': '#6c757d', 'color': 'white',
                                   'border': 'none', 'padding': '10px 20px', 'border-radius': '5px', 'cursor': 'pointer', 'margin-right': '10px'}),
                html.Button("Next", id='next-button', n_clicks=0,
                            style={'margin-top': '20px', 'background-color': '#007bff', 'color': 'white',
                                   'border': 'none', 'padding': '10px 20px', 'border-radius': '5px', 'cursor': 'pointer'}),
            ]
        )
    ]
)

@callback(
    [Output('step-container', 'children'),
     Output('step-store', 'data')],  # Update both the container and the stored step
    [Input('next-button', 'n_clicks'),
     Input('prev-button', 'n_clicks')],
    [State('step-store', 'data')],  # Get current step from store
    prevent_initial_call=True,  # Prevent callback on initial load
    allow_duplicate=True  # Allow duplicate outputs
)
def update_step(next_clicks, prev_clicks, current_step):
    ctx = dash.callback_context

    # If no button clicked yet, start at step 0
    if not ctx.triggered:
        current_step = 0
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if button_id == 'next-button' and current_step < len(steps) - 1:
            current_step += 1
        elif button_id == 'prev-button' and current_step > 0:
            current_step -= 1

    step = steps[current_step]

    # Render text or plot based on step type
    if step['type'] == 'text':
        content = [
            html.Div(
                style={'padding': '20px', 'background-color': '#f8f8f8', 'border-radius': '8px',
                       'box-shadow': '2px 2px 5px rgba(0,0,0,0.1)'},
                children=[
                    html.H2(step['title'], style={'color': '#333'}),
                    html.P(step['content'], style={'font-size': '16px', 'color': '#555'})
                ]
            )
        ]
    elif step['type'] == 'plot':
        if current_step == 1:  # Bar chart for trend
            fig = px.bar(df, x='Year', y='Activity', title=step['title'],
                         labels={'Activity': f'{category} Activity'},
                         color='Activity', color_continuous_scale='Viridis')
        else:  # Pie chart for distribution
            fig = px.pie(df, names='Year', values='Activity', title=step['title'],
                         color_discrete_sequence=px.colors.sequential.RdBu)

        content = [
            html.Div(
                style={'width': '500px', 'text-align': 'center', 'margin': 'auto'},
                children=[
                    html.H3(step['content'], style={'color': '#333'}),
                    dcc.Graph(figure=fig, style={'width': '100%', 'height': '400px'})
                ]
            )
        ]

    return content, current_step  # Return both the content and updated step
import pandas as pd

# Load the data
df = pd.read_csv("personalized_data.csv")
user_name = df['User'].iloc[0]
category = df['Category'].iloc[0]

# Simulate a zero-shot LLM insight (in practice, you'd use an API like Hugging Face or xAI's API)
def get_insight(df, category, user_name):
    max_year = df.loc[df['Activity'].idxmax(), 'Year']
    max_activity = df['Activity'].max()
    insight = (
        f"Hey {user_name}, your {category} activity peaked in {int(max_year)} "
        f"with a value of {max_activity:.1f}! Looks like that was your golden year—maybe you "
        f"mastered {category} or just had a lot of fun. What do you think happened?"
    )
    return insight

# Generate a zero-shot LLM insight using a Hugging Face model
def get_llm_insight(df, category, user_name):
    max_year = df.loc[df['Activity'].idxmax(), 'Year']
    max_activity = df['Activity'].max()
    insight = (
        f"Hey {user_name}, your {category} activity peaked in {int(max_year)} "
        f"with a value of {max_activity:.1f}! Looks like that was your golden year—maybe you "
        f"mastered {category} or just had a lot of fun. What do you think happened?"
    )
    return insight

# Get the insight
llm_insight = get_insight(df, category, user_name)

# Add the insight to the Wrapped steps (append this to the `steps` list in Phase 2)
new_step = {
    'type': 'text',
    'title': f'🔮 Magic Insight for {user_name} 🔮',
    'content': llm_insight
}
steps.append(new_step)
final_step =  {
        'type': 'text',
        'title': '🔮 That’s a Wrap! 🔮',
        'content': f'Thanks for exploring your {category} journey, {user_name}!'
    }
steps.append(final_step)

In [12]:
if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>

And that concludes our quick tutorial!